In [ ]:

!pip install gdown --quiet
!pip install --upgrade transformers datasets scikit-learn pandas pyarrow --quiet


--- LIBRARIES UPGRADED ---
==> STEP 1 IS DONE. <==

==> IMPORTANT: YOU MUST RESTART THE RUNTIME NOW. <==
Go to the menu: 'Runtime' -> 'Restart runtime'.
After restarting, you can run all the *other* cells (Step 2, 3, 4, 5).


In [ ]:
import gdown
import os

file_id = '1403EGqzIDoHMdQF4c9Bkyl7dZLZ5Wt6J'
output = 'spider.zip'

print("Downloading Spider dataset from Google Drive...")
if not os.path.exists(output):
    gdown.download(id=file_id, output=output, quiet=False)
else:
    print("spider.zip already downloaded.")

print("Unzipping files...")
!unzip -o spider.zip -d .

print("\n--- Checking files ---")
print("Files in current directory:")
!ls -l
print("\nFiles in 'spider' directory:")
!ls -l spider
print("------------------------")

spider.zip already downloaded.
Unzipping files...
Archive:  spider.zip
  inflating: ./spider_data/dev_gold.sql  
  inflating: ./__MACOSX/spider_data/._dev_gold.sql  
  inflating: ./__MACOSX/spider_data/._database  
  inflating: ./spider_data/.DS_Store  
  inflating: ./__MACOSX/spider_data/._.DS_Store  
  inflating: ./spider_data/test_tables.json  
  inflating: ./__MACOSX/spider_data/._test_tables.json  
  inflating: ./spider_data/train_others.json  
  inflating: ./__MACOSX/spider_data/._train_others.json  
  inflating: ./spider_data/train_spider.json  
  inflating: ./__MACOSX/spider_data/._train_spider.json  
  inflating: ./spider_data/test.json  
  inflating: ./__MACOSX/spider_data/._test.json  
  inflating: ./spider_data/tables.json  
  inflating: ./__MACOSX/spider_data/._tables.json  
  inflating: ./spider_data/dev.json  
  inflating: ./__MACOSX/spider_data/._dev.json  
  inflating: ./__MACOSX/spider_data/._test_database  
  inflating: ./spider_data/README.txt  
  inflating: ./__MAC

In [ ]:
import pandas as pd
import numpy as np


path_train_spider = 'spider_data/train_spider.json'
path_train_others = 'spider_data/train_others.json' 
path_test = 'spider_data/dev.json'


try:
    df_spider = pd.read_json(path_train_spider)
    df_others = pd.read_json(path_train_others)
    test_df = pd.read_json(path_test)

    train_df = pd.concat([df_spider, df_others], ignore_index=True)

    print(f"Successfully loaded {len(train_df)} training samples and {len(test_df)} test samples.")
except FileNotFoundError:
    print(f"ERROR: Cannot find files. Please ensure 'spider_data' directory is correct.")

def get_difficulty_v2(sql):
    sql_upper = str(sql).upper()
    select_count = sql_upper.count('SELECT')

    if select_count > 2 or 'INTERSECT' in sql_upper or 'UNION' in sql_upper or 'EXCEPT' in sql_upper:
        return 3

    if select_count == 2 or 'HAVING' in sql_upper:
        return 2

    if 'GROUP BY' in sql_upper or 'ORDER BY' in sql_upper or 'JOIN' in sql_upper or 'LIMIT' in sql_upper:
        return 1

    return 0

train_df['label'] = train_df['query'].apply(get_difficulty_v2)
test_df['label'] = test_df['query'].apply(get_difficulty_v2)

train_df = train_df[['question', 'label']]
test_df = test_df[['question', 'label']]

X_train = train_df['question']
y_train = train_df['label']
X_test = test_df['question']
y_test = test_df['label']

print("\nData is ready (with v2 labels):")
print(train_df.head())
print("\nNew label distribution in training data:")
print(train_df['label'].value_counts().sort_index())

Successfully loaded 8659 training samples and 1034 test samples.

Data is ready (with v2 labels):
                                            question  label
0  How many heads of the departments are older th...      0
1  List the name, born state and age of the heads...      1
2  List the creation year, name and budget of eac...      0
3  What are the maximum and minimum budget of the...      0
4  What is the average number of employees of the...      0

New label distribution in training data:
label
0    2377
1    4525
2    1158
3     599
Name: count, dtype: int64


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score

baseline_model = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000, ngram_range=(1, 2))),
    ('svm', SVC(kernel='linear'))
])

print("Training baseline model...")
baseline_model.fit(X_train, y_train)

print("Evaluating baseline model...")
baseline_preds = baseline_model.predict(X_test)

baseline_accuracy = accuracy_score(y_test, baseline_preds)
baseline_f1 = f1_score(y_test, baseline_preds, average='macro')

print("\n--- BASELINE MODEL RESULTS ---")
print(f"Accuracy: {baseline_accuracy * 100:.2f}%")
print(f"Macro F1-Score: {baseline_f1 * 100:.2f}%")
print("----------------------------------")

Training baseline model...
Evaluating baseline model...

--- BASELINE MODEL RESULTS ---
Accuracy: 76.89%
Macro F1-Score: 74.48%
----------------------------------

==> WRITE THESE TWO NUMBERS DOWN FOR YOUR PAPER! <==


In [ ]:
import numpy as np
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
from sklearn.metrics import accuracy_score, f1_score
import os

os.environ["WANDB_DISABLED"] = "true"

train_dataset = Dataset.from_pandas(train_df[['question', 'label']])
test_dataset = Dataset.from_pandas(test_df[['question', 'label']])

print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples['question'], padding="max_length", truncation=True)

print("Tokenizing datasets...")
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

print("Loading pre-trained DB-BERT model...")
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=4,
    id2label={0: 'easy', 1: 'medium', 2: 'hard', 3: 'extra_hard'},
    label2id={'easy': 0, 'medium': 1, 'hard': 2, 'extra_hard': 3} 
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='macro')
    return {"accuracy": acc, "f1-macro": f1}

print("Setting up training arguments...")

training_args = TrainingArguments(
    output_dir="test_trainer",
    num_train_epochs=4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    report_to="none", # This disables the wandb popup
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,  # We still give it the eval dataset
    compute_metrics=compute_metrics,    # We still tell it how to compute metrics
)

# 9. TRAIN THE MODEL!
print("\nTraining DB-BERT model... ")
# This will just train for 4 epochs, no evaluation in the middle.
trainer.train()

print("Evaluating DB-BERT model...")
eval_results = trainer.evaluate()

print("\n--- DB-BERT MODEL RESULTS ---")
print(f"Accuracy: {eval_results['eval_accuracy'] * 100:.2f}%")
print(f"Macro F1-Score: {eval_results['eval_f1-macro'] * 100:.2f}%")
print("-------------------------------")

Loading tokenizer...
Tokenizing datasets...


Map:   0%|          | 0/8659 [00:00<?, ? examples/s]

Map:   0%|          | 0/1034 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loading pre-trained DB-BERT model...
Setting up training arguments...

Training DB-BERT model... (This will take 10-15 minutes)


Step,Training Loss
500,0.600700
1000,0.272200
1500,0.133900
2000,0.053400


Evaluating DB-BERT model...



--- DB-BERT MODEL RESULTS ---
Accuracy: 81.62%
Macro F1-Score: 81.02%
-------------------------------

==> WRITE THESE TWO NUMBERS DOWN FOR YOUR PAPER! <==
